In [ ]:
a = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']

In [ ]:
import pandas as pd
import os
import cv2
import argparse
from typing import Tuple
import albumentations as A
import pandas as pd
import torch
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn, optim
from torchvision import datasets, transforms, models
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.nn import functional as F
from PIL import Image


In [ ]:
def plot_samples(df, data_root_path, category, num_imgs=20, num_cols=5):
    
    paths = df[df.category == category].file_path.sample(num_imgs)
    
    num_rows = num_imgs // num_cols 
    if num_imgs % num_cols != 0:
        num_rows += 1
    
    fig = plt.figure(figsize=(num_cols * 4, num_rows * 4))
    for idx, path in enumerate(paths):

        ax = fig.add_subplot(num_rows,num_cols,idx+1)
        
        im = cv2.imread(os.path.join(data_root_path, path))
        im_resized = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)

        plt.imshow(cv2.cvtColor(im_resized, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
        
    plt.show()
# for category in sorted(train_df.category.unique()):
#     print(category)
#     plot_samples(train_df, TRAIN_PATH, category, 5, 5)

In [ ]:
class ResNetBase(nn.Module):
    def __init__(self, backbone='resnext50_32x4d_ssl'):
        super(ResNetBase, self).__init__()
        if backbone.endswith('l'):
            self.backbone = torch.hub.load(
                'facebookresearch/semi-supervised-ImageNet1K-models',
                backbone,
            )
        else:
            self.backbone = getattr(models, backbone)(pretrained=True)
        self.out_features = self.backbone.fc.in_features

    def forward(self, x):
        base = self.backbone
        x = base.conv1(x)
        x = base.bn1(x)
        x = base.relu(x)
        x = base.maxpool(x)

        x = base.layer1(x)
        x = base.layer2(x)
        x = base.layer3(x)
        x = base.layer4(x)
        return x

In [ ]:
class ResNetHead(nn.Module):
    def __init__(self, in_features: int, n_classes: int, use_neck: bool):
        super().__init__()

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(in_features, n_classes)
        self.use_neck = use_neck

    def forward(self, x):
        if not self.use_neck:
            x = self.pooling(x)
            x = torch.flatten(x, start_dim=1)
        x = self.apply_fc_out(x)
        return x

    def apply_fc_out(self, x):
        return self.fc1(x)


In [ ]:
class Neck(nn.Module):
    def __init__(self, in_features: int, hidden_dim):
        super().__init__()

        self.pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.bn1 = nn.BatchNorm1d(in_features)
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)

    def forward(self, x):

        x = self.pooling(x)
        x = torch.flatten(x, start_dim=1)
        x = self.bn1(x)
        x = F.relu(self.fc1(x))
        x = self.bn2(x)
        x = self.fc2(x)
        x = self.bn3(x)

        return x

In [ ]:
def build_model(backbone: str, n_classes: int, **kwargs) -> nn.Module:
    return Model(backbone=backbone, n_classes=n_classes, **kwargs)


class Model(nn.Module):
    def __init__(self, *, backbone: str, n_classes: int, use_neck: bool,):
        super().__init__()

        self.backbone = ResNetBase(backbone)
        self.in_features = self.backbone.out_features
        self.use_neck = use_neck
        if self.use_neck:
            self.hidden_dim = 1024
            self.neck = Neck(self.in_features, 1024)
            self.in_features = self.hidden_dim
        self.head = ResNetHead(self.in_features, n_classes, self.use_neck)

    def forward(self, x):
        x = self.backbone(x)
        if self.use_neck:
            x = self.neck(x)
        x = self.head(x)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
DATA_PATH = "/kaggle/input/shopee-code-league-20/_DA_Product_Detection"

TRAIN_CSV_PATH = os.path.join(DATA_PATH, "train.csv")
TEST_CSV_PATH = os.path.join(DATA_PATH, "test.csv")

TRAIN_PATH = os.path.join(DATA_PATH, "train", "train")
TEST_PATH = os.path.join(DATA_PATH, "test", "test")

train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df = pd.read_csv(TEST_CSV_PATH)

display(train_df.head())
display(test_df.head())

image_size = 256
batch_size = 64

In [ ]:
train_str_category = train_df.category.apply(lambda x: str(x).zfill(2))
train_df['file_path'] = train_str_category + os.sep + train_df.filename

test_str_category = test_df.category.apply(lambda x: str(x).zfill(2))
test_df['file_path'] = test_df.filename

display(train_df)
display(test_df)

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       ])

test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      ])

In [ ]:
train_set = datasets.ImageFolder(TRAIN_PATH,       
                    transform=train_transforms)

split_idx = int(train_df.shape[0] / 5 * 4)

train_set, val_set = torch.utils.data.random_split(train_set, [split_idx, train_df.shape[0] - split_idx])


test_set = datasets.ImageFolder(os.path.join(TEST_PATH, ".."),       
                    transform=test_transforms)

In [ ]:
train_set_subset = torch.utils.data.Subset(train_set, np.random.choice(len(train_set), 160, replace=False))
val_set_subset = torch.utils.data.Subset(val_set, np.random.choice(len(val_set), 40, replace=False))
test_set_subset = torch.utils.data.Subset(test_set, np.random.choice(len(test_set), 40, replace=False))

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set_subset, batch_size=20, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set_subset, batch_size=20)
test_loader = torch.utils.data.DataLoader(test_set_subset, batch_size=1)

In [ ]:
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_set, batch_size=64)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
figure_dict ={
    "train_loss":[],
    "val_loss":[],
    "train_acc":[],
    "val_acc":[]
}


In [ ]:
train_losses = []
train_accu = []
def train_epoch(
        loader, model, device,
        criterion, optimizer, 
        n_classes,
):
    global train_accu, train_losses
    correct=0
    total=0
    running_loss=0
    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device, dtype=torch.float), target.to(device)
        
        optimizer.zero_grad()
        preds = model(data)
        loss = criterion(preds, target)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        _, predicted = preds.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description(f'loss: {loss_np:.5f}, smth: {smooth_loss:.5f}')
    accu=100.*correct/total
    train_loss=running_loss/len(loader)
    train_accu.append(accu)
    train_losses.append(train_loss)    
    if epoch % 8 == 0:
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, "model_{}.pth".format(epoch))
    global figure_dict
    figure_dict["train_loss"].append(train_losses)
    

In [ ]:
eval_losses=[]
eval_accu=[]
def val_epoch(loader, model, device, criterion, n_classes):
    global   eval_losses,eval_accu
    val_loss = 0
    acc = 0
   # top_acc = TopKAccuracy(k=1)

    model.eval()
    with torch.no_grad():
        for (data, targets) in tqdm(loader):
            data, targets = data.to(device, dtype=torch.float), targets.to(device)
            
            preds = model(data)
            loss = criterion(preds, targets)

            val_loss += loss.item()
            acc += (preds.argmax(1) == targets).float().mean().item()

    #        top_acc.update(targets, preds)

    n_total = len(loader)
    val_loss = val_loss / n_total
    acc = acc / n_total
    eval_losses.append(val_loss)
    eval_accu.append(acc)
    #top_acc = top_acc.compute()
    global firgure_dict
    figure_dict["val_loss"].append(val_loss)
    figure_dict["val_acc"].append(acc)
    print('acc', acc)
    return val_loss, acc

In [ ]:
model = build_model(
        backbone='resnext50_32x4d_ssl', n_classes=42,
        use_neck=0,
    )
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer.zero_grad()
optimizer.step()
criterion_fn = nn.CrossEntropyLoss()
n_classes=42

In [ ]:
for epoch in range(1, 6):
#    print(time.ctime(), 'Epoch:', epoch)
#    scheduler.step(epoch - 1)
    train_loss = train_epoch(
        train_loader, model,
        device, criterion_fn, optimizer,
        n_classes,
    )
    val_loss, acc = val_epoch(
        val_loader, model,
        device, criterion_fn,
        n_classes,
    )


In [ ]:
for key, value in figure_dict.items():
    print(key, ' : ', value)

In [ ]:
plt.plot(train_accu,'-o')
plt.plot(eval_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')
 
plt.show()

In [ ]:
print(eval_accu)
print(train_accu)

In [ ]:
plt.plot(train_losses,'-o')
plt.plot(eval_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')
 
plt.show()

In [ ]:
print(train_losses)
print(eval_losses)